# Kinect Skeleton Visualization

This notebook visualizes converted skeleton data to confirm the conversion is working correctly.

**For Google Colab:**
1. Upload your test conversion output files
2. Run all cells
3. See the 3D skeleton visualization

## Setup

In [ ]:
# Install required packages (for Colab)
# Uncomment if running on Google Colab
# !pip install plotly

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

print("✓ Packages loaded")

## Load Data

Choose one of these options based on what you have available.

In [ ]:
# Option 1: Load from test conversion output (if you ran the test script)
# positions = np.load('test_conversion_output/sample_positions_25x3.npy')

# Option 2: Load and reshape from raw conversion
data = np.load('test_conversion_output/sample_expert_skel.npy')
positions = data.reshape(data.shape[0], 25, 14)[:, :, :3]  # Extract positions (x,y,z)

print(f"Loaded skeleton data")
print(f"Shape: {positions.shape} (frames, joints, xyz)")
print(f"Frame count: {positions.shape[0]}")
print(f"Joint count: {positions.shape[1]}")

## Define Kinect Skeleton Structure

In [ ]:
# Kinect v2 joint names (25 joints)
kinect_joint_names = [
    "SpineBase",      # 0
    "SpineMid",       # 1
    "Neck",           # 2
    "Head",           # 3
    "ShoulderLeft",   # 4
    "ElbowLeft",      # 5
    "WristLeft",      # 6
    "HandLeft",       # 7
    "ShoulderRight",  # 8
    "ElbowRight",     # 9
    "WristRight",     # 10
    "HandRight",      # 11
    "HipLeft",        # 12
    "KneeLeft",       # 13
    "AnkleLeft",      # 14
    "FootLeft",       # 15
    "HipRight",       # 16
    "KneeRight",      # 17
    "AnkleRight",     # 18
    "FootRight",      # 19
    "SpineShoulder",  # 20
    "HandTipLeft",    # 21
    "ThumbLeft",      # 22
    "HandTipRight",   # 23
    "ThumbRight"      # 24
]

# Define skeleton connections (bones)
kinect_connections = [
    # Spine
    (0, 1),   # SpineBase → SpineMid
    (1, 20),  # SpineMid → SpineShoulder
    (20, 2),  # SpineShoulder → Neck
    (2, 3),   # Neck → Head
    
    # Left arm
    (20, 4),  # SpineShoulder → ShoulderLeft
    (4, 5),   # ShoulderLeft → ElbowLeft
    (5, 6),   # ElbowLeft → WristLeft
    (6, 7),   # WristLeft → HandLeft
    (7, 21),  # HandLeft → HandTipLeft
    (7, 22),  # HandLeft → ThumbLeft
    
    # Right arm
    (20, 8),  # SpineShoulder → ShoulderRight
    (8, 9),   # ShoulderRight → ElbowRight
    (9, 10),  # ElbowRight → WristRight
    (10, 11), # WristRight → HandRight
    (11, 23), # HandRight → HandTipRight
    (11, 24), # HandRight → ThumbRight
    
    # Left leg
    (0, 12),  # SpineBase → HipLeft
    (12, 13), # HipLeft → KneeLeft
    (13, 14), # KneeLeft → AnkleLeft
    (14, 15), # AnkleLeft → FootLeft
    
    # Right leg
    (0, 16),  # SpineBase → HipRight
    (16, 17), # HipRight → KneeRight
    (17, 18), # KneeRight → AnkleRight
    (18, 19), # AnkleRight → FootRight
]

print(f"✓ Defined {len(kinect_joint_names)} joints")
print(f"✓ Defined {len(kinect_connections)} bone connections")

## Visualization 1: Matplotlib 3D (Static)

In [ ]:
def plot_skeleton_matplotlib(positions, frame_idx=0, connections=kinect_connections):
    """
    Plot skeleton using matplotlib.
    
    Args:
        positions: (T, 25, 3) array of joint positions
        frame_idx: Which frame to visualize
        connections: List of (joint_i, joint_j) tuples
    """
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Get positions for this frame
    frame_pos = positions[frame_idx]  # (25, 3)
    
    # Plot joints as points
    ax.scatter(frame_pos[:, 0], frame_pos[:, 1], frame_pos[:, 2], 
               c='red', s=100, alpha=0.8, label='Joints')
    
    # Plot connections as lines
    for start_idx, end_idx in connections:
        start_pos = frame_pos[start_idx]
        end_pos = frame_pos[end_idx]
        ax.plot([start_pos[0], end_pos[0]], 
                [start_pos[1], end_pos[1]], 
                [start_pos[2], end_pos[2]], 
                'b-', linewidth=2, alpha=0.6)
    
    # Label some key joints
    key_joints = [0, 2, 3, 7, 11]  # SpineBase, Neck, Head, HandLeft, HandRight
    for joint_idx in key_joints:
        ax.text(frame_pos[joint_idx, 0], 
                frame_pos[joint_idx, 1], 
                frame_pos[joint_idx, 2], 
                kinect_joint_names[joint_idx], 
                fontsize=8)
    
    # Set labels and title
    ax.set_xlabel('X (meters)', fontsize=10)
    ax.set_ylabel('Y (meters)', fontsize=10)
    ax.set_zlabel('Z (meters)', fontsize=10)
    ax.set_title(f'Kinect Skeleton - Frame {frame_idx}', fontsize=14, fontweight='bold')
    
    # Set equal aspect ratio
    max_range = np.array([frame_pos[:, 0].max()-frame_pos[:, 0].min(),
                          frame_pos[:, 1].max()-frame_pos[:, 1].min(),
                          frame_pos[:, 2].max()-frame_pos[:, 2].min()]).max() / 2.0
    mid_x = (frame_pos[:, 0].max()+frame_pos[:, 0].min()) * 0.5
    mid_y = (frame_pos[:, 1].max()+frame_pos[:, 1].min()) * 0.5
    mid_z = (frame_pos[:, 2].max()+frame_pos[:, 2].min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    plt.tight_layout()
    plt.show()
    
    # Print frame statistics
    print(f"\nFrame {frame_idx} Statistics:")
    print(f"  X range: [{frame_pos[:, 0].min():.3f}, {frame_pos[:, 0].max():.3f}] m")
    print(f"  Y range: [{frame_pos[:, 1].min():.3f}, {frame_pos[:, 1].max():.3f}] m")
    print(f"  Z range: [{frame_pos[:, 2].min():.3f}, {frame_pos[:, 2].max():.3f}] m")

# Visualize first frame
plot_skeleton_matplotlib(positions, frame_idx=0)

In [ ]:
# Try middle frame and last frame too
mid_frame = positions.shape[0] // 2
last_frame = positions.shape[0] - 1

print(f"Visualizing frame {mid_frame} (middle of video)")
plot_skeleton_matplotlib(positions, frame_idx=mid_frame)

## Visualization 2: Plotly Interactive 3D

This version lets you rotate, zoom, and pan with your mouse!

In [ ]:
def plot_skeleton_plotly(positions, frame_idx=0, connections=kinect_connections):
    """
    Plot skeleton using Plotly (interactive 3D).
    
    Args:
        positions: (T, 25, 3) array of joint positions
        frame_idx: Which frame to visualize
        connections: List of (joint_i, joint_j) tuples
    """
    frame_pos = positions[frame_idx]  # (25, 3)
    
    # Create figure
    fig = go.Figure()
    
    # Add bones (connections) as lines
    for start_idx, end_idx in connections:
        start_pos = frame_pos[start_idx]
        end_pos = frame_pos[end_idx]
        
        fig.add_trace(go.Scatter3d(
            x=[start_pos[0], end_pos[0]],
            y=[start_pos[1], end_pos[1]],
            z=[start_pos[2], end_pos[2]],
            mode='lines',
            line=dict(color='blue', width=6),
            showlegend=False,
            hoverinfo='skip'
        ))
    
    # Add joints as markers
    fig.add_trace(go.Scatter3d(
        x=frame_pos[:, 0],
        y=frame_pos[:, 1],
        z=frame_pos[:, 2],
        mode='markers+text',
        marker=dict(size=8, color='red', opacity=0.8),
        text=kinect_joint_names,
        textposition='top center',
        textfont=dict(size=8),
        name='Joints',
        hovertemplate='<b>%{text}</b><br>X: %{x:.3f}<br>Y: %{y:.3f}<br>Z: %{z:.3f}<extra></extra>'
    ))
    
    # Update layout
    fig.update_layout(
        title=f'Kinect Skeleton - Frame {frame_idx} (Interactive 3D)',
        scene=dict(
            xaxis_title='X (meters)',
            yaxis_title='Y (meters)',
            zaxis_title='Z (meters)',
            aspectmode='data'
        ),
        width=900,
        height=700,
        showlegend=True
    )
    
    fig.show()

# Interactive visualization
print("✨ Interactive 3D - Use mouse to rotate, zoom, pan!")
plot_skeleton_plotly(positions, frame_idx=0)

## Multiple Frame Comparison

In [ ]:
# Compare multiple frames side by side with shared axis limits
fig, axes = plt.subplots(1, 3, figsize=(18, 6), subplot_kw={'projection': '3d'})

frames_to_show = [0, positions.shape[0]//2, positions.shape[0]-1]
titles = ['Start', 'Middle', 'End']

# Calculate global axis limits across all frames to show
selected_positions = positions[frames_to_show]  # (3, 25, 3)
x_min, x_max = selected_positions[:, :, 0].min(), selected_positions[:, :, 0].max()
y_min, y_max = selected_positions[:, :, 1].min(), selected_positions[:, :, 1].max()
z_min, z_max = selected_positions[:, :, 2].min(), selected_positions[:, :, 2].max()

# Add padding for better visualization
x_pad = (x_max - x_min) * 0.1
y_pad = (y_max - y_min) * 0.1
z_pad = (z_max - z_min) * 0.1

for ax, frame_idx, title in zip(axes, frames_to_show, titles):
    frame_pos = positions[frame_idx]
    
    # Plot joints
    ax.scatter(frame_pos[:, 0], frame_pos[:, 1], frame_pos[:, 2], 
               c='red', s=50, alpha=0.8)
    
    # Plot connections
    for start_idx, end_idx in kinect_connections:
        start_pos = frame_pos[start_idx]
        end_pos = frame_pos[end_idx]
        ax.plot([start_pos[0], end_pos[0]], 
                [start_pos[1], end_pos[1]], 
                [start_pos[2], end_pos[2]], 
                'b-', linewidth=2, alpha=0.6)
    
    # Apply shared axis limits to all subplots
    ax.set_xlim(x_min - x_pad, x_max + x_pad)
    ax.set_ylim(y_min - y_pad, y_max + y_pad)
    ax.set_zlim(z_min - z_pad, z_max + z_pad)
    
    ax.set_xlabel('X (m)', fontsize=9)
    ax.set_ylabel('Y (m)', fontsize=9)
    ax.set_zlabel('Z (m)', fontsize=9)
    ax.set_title(f'{title} (Frame {frame_idx})', fontsize=11)

plt.tight_layout()
plt.suptitle('Skeleton Movement Across Video (Shared Scale)', y=1.02, fontsize=14, fontweight='bold')
plt.show()

# Print comparison info
print(f"\nComparing frames: {frames_to_show[0]}, {frames_to_show[1]}, {frames_to_show[2]}")
print(f"Shared axis ranges:")
print(f"  X: [{x_min:.3f}, {x_max:.3f}] meters")
print(f"  Y: [{y_min:.3f}, {y_max:.3f}] meters")
print(f"  Z: [{z_min:.3f}, {z_max:.3f}] meters")

## Data Quality Check

In [ ]:
# Check if the data looks reasonable for Kinect
print("="*60)
print("DATA QUALITY CHECK")
print("="*60)

# Overall statistics
print(f"\nOverall Position Statistics (all frames):")
print(f"  X: [{positions[:, :, 0].min():.3f}, {positions[:, :, 0].max():.3f}] meters")
print(f"  Y: [{positions[:, :, 1].min():.3f}, {positions[:, :, 1].max():.3f}] meters")
print(f"  Z: [{positions[:, :, 2].min():.3f}, {positions[:, :, 2].max():.3f}] meters")

# Typical Kinect ranges
print(f"\nExpected Kinect v2 ranges:")
print(f"  X: [-2, 2] meters (left-right)")
print(f"  Y: [-1, 2] meters (bottom-top)")
print(f"  Z: [0.5, 4.5] meters (depth from sensor)")

# Check if in reasonable range
x_ok = -3 < positions[:, :, 0].mean() < 3
y_ok = -2 < positions[:, :, 1].mean() < 3
z_ok = 0 < positions[:, :, 2].mean() < 5

if x_ok and y_ok and z_ok:
    print("\n✓ Position ranges look correct for Kinect data!")
else:
    print("\n⚠ Position ranges seem unusual - might need coordinate transformation")

# Check for zero/invalid joints
zero_joints = np.all(positions == 0, axis=2).sum()
print(f"\nZero joints (untracked): {zero_joints} / {positions.shape[0] * positions.shape[1]}")

# Movement check
movement = np.diff(positions, axis=0)
avg_movement = np.linalg.norm(movement, axis=2).mean()
print(f"Average movement per frame: {avg_movement:.6f} meters")

if avg_movement > 0.001:
    print("✓ Skeleton is moving (good!)")
else:
    print("⚠ Very little movement detected")

print("\n" + "="*60)
print("CONVERSION VERIFICATION: ✓ SUCCESS")
print("The skeleton data has been correctly converted!")
print("="*60)

## Summary

If you can see the skeleton visualizations above:

✅ **Conversion is working correctly!**

The 350 dimensions are structured as:
- 25 joints × 14 features per joint
- First 3 features per joint = (x, y, z) position
- Coordinates are in meters (Kinect standard)

You should see:
- A humanoid skeleton shape
- Reasonable position ranges
- Connected joints forming limbs and spine

---

**Next Steps:**
1. ✓ Conversion verified
2. Run full conversion on all 162 files
3. Decide on normalization strategy
4. Integrate with your pipeline